Nathaniel's Branch

In [1]:
# Initalize GPU
import torch
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os 
%cd /content/drive/MyDrive/CAIS_Sentiment140_Project/
if not ( os.path.exists("/content/drive/MyDrive/CAIS_Sentiment140_Project/training.1600000.processed.noemoticon.csv") ):
  !unzip trainingandtestdata.zip # type: ignore
  %ls

/content/drive/MyDrive/CAIS_Sentiment140_Project


In [4]:
import pandas as pd
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text' ])
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 1,600,000



In [5]:
# Display 10 random rows from the data.
df.sample(10)

,target,ids,date,flag,user,text
1576353,4,2189525183,Tue Jun 16 00:14:23 PDT 2009,NO_QUERY,germancookiemon,Should be a half work-day today only. Let's see!
354896,0,2039856597,Thu Jun 04 22:59:13 PDT 2009,NO_QUERY,Killaya,"@SmashMe_EraseMe I'm sorry babe, what's wrong?"
669794,0,2246363962,Fri Jun 19 17:29:11 PDT 2009,NO_QUERY,snowflake327,@djhappy115 awe wish I was there
750029,0,2285378158,Mon Jun 22 15:03:22 PDT 2009,NO_QUERY,kevn182,"@sims then dont look on twitter, i was spoiled..."
1478996,4,2066589902,Sun Jun 07 10:48:22 PDT 2009,NO_QUERY,nikkibee,Omg tummy hurts- none the less on my way to th...
1056272,4,1962439207,Fri May 29 11:02:54 PDT 2009,NO_QUERY,astridnathania,just called by my bf huhu . .how i love his v...
1524972,4,2176797117,Mon Jun 15 04:46:35 PDT 2009,NO_QUERY,KriisTiincHen,SchOol was funny IÂ´m tired =(
209722,0,1974048321,Sat May 30 12:20:15 PDT 2009,NO_QUERY,Sgt_pepper_girl,@lovelyrita_mm i'm sorry it hurt so much. And...
86990,0,1754308477,Sun May 10 05:48:39 PDT 2009,NO_QUERY,JP_waste2energy,Re-installed OS-X on macbook after a broken bi...
154115,0,1933098636,Tue May 26 22:59:46 PDT 2009,NO_QUERY,heidiprice,@BudgetBitch bugger. some people are so shorts...


In [6]:
print("0, negative sentiment")
df.loc[df.target == 0].sample(5)[['target', 'text']]

0, negative sentiment


,target,text
227659,0,@mangopickle i like stab movies.... idk im sc...
216721,0,@HeyErnie lame lol ha ha ha j leno is leaving...
607283,0,me and arron say meannnnyyy
324799,0,I'm not saying anything naughty and I'm not se...
77377,0,is doing Monday morning paperwork...blah


In [7]:
print("4, positive sentiment")
df.loc[df.target == 4].sample(5)[['target', 'text']]

4, positive sentiment


,target,text
993280,4,@anxiousdog Me too - I woke up about an hour e...
859445,4,Its offically May 2nd xD..12:00 AM
1298358,4,and also: gay man pushing iddy biddy black boy...
1146489,4,@ashleymarfil you are welcome! Im loving the g...
822496,4,going out to eat with samantha tavares!!!!! ...


In [8]:
#Extract text and target labels
texts = df.text.values
targets = df.target.values

In [9]:
!pip install transformers
import transformers
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 10.1 MB/s 
     |████████████████████████████████| 7.6 MB 60.0 MB/s 
     |████████████████████████████████| 182 kB 74.6 MB/s 
Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

#Potential To-Do = Further Scrub Data of mentions, links, and other punctuation

In [10]:
# Print the original sentence.
print(' Original: ', texts[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(texts[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[0])))

 Original:  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Tokenized:  ['@', 'switch', '##foot', 'http', ':', '/', '/', 't', '##wi', '##tp', '##ic', '.', 'com', '/', '2', '##y', '##1', '##z', '##l', '-', 'aw', '##w', '##w', ',', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', ';', 'd']
Token IDs:  [1030, 6942, 13064, 8299, 1024, 1013, 1013, 1056, 9148, 25856, 2594, 1012, 4012, 1013, 1016, 2100, 2487, 2480, 2140, 1011, 22091, 2860, 2860, 1010, 2008, 1005, 1055, 1037, 26352, 5017, 1012, 2017, 2323, 2050, 2288, 2585, 12385, 1997, 2353, 2154, 2000, 2079, 2009, 1012, 1025, 1040]


In [11]:
if os.path.exists("/content/drive/MyDrive/CAIS_Sentiment140_Project/training.1600000.processed.noemoticon.csv"):
  input_ids = torch.load("/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/input_ids.pt")
  attention_masks = torch.load("/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/attention_masks.pt")
  targets = torch.load("/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/targets.pt")
else:
  input_ids = []
  attention_masks = []

  # For every sentence...
  for t in texts:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          t,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 140,           # Pad & truncate all sentences.
                          pad_to_max_length = True,          # Pad & truncate all sentences.
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt'     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  targets = torch.tensor(targets)

  # Print sentence 0, now as a list of IDs.
  print('Original: ', texts[0])
  print('Token IDs:', input_ids[0])

In [12]:
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])
print('Attention Masks', attention_masks[0])
print("Targets", targets[0])

Original:  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Token IDs: tensor([  101,  1030,  6942, 13064,  8299,  1024,  1013,  1013,  1056,  9148,
        25856,  2594,  1012,  4012,  1013,  1016,  2100,  2487,  2480,  2140,
         1011, 22091,  2860,  2860,  1010,  2008,  1005,  1055,  1037, 26352,
         5017,  1012,  2017,  2323,  2050,  2288,  2585, 12385,  1997,  2353,
         2154,  2000,  2079,  2009,  1012,  1025,  1040,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
    

In [13]:
if os.path.exists("/content/drive/MyDrive/CAIS_Sentiment140_Project/training.1600000.processed.noemoticon.csv"):
  print("Hey, It's Joey here.")
else:
  newpath = r'/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors' 
  if not os.path.exists(newpath):
      os.makedirs(newpath)

  torch.save(texts, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/texts.pt" )
  torch.save(input_ids, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/input_ids.pt" )
  torch.save(attention_masks, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/attention_masks.pt" )
  torch.save(targets, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/targets.pt" )

Hey, It's Joey here.


#Word Embedding Step Done

In [31]:
#Test Train Val Split

##### CHANGED FROM 0.95


from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, targets)
train_size = int(0.20 * len(dataset))
test_size = int(0.70 * len(dataset))
val_size = len(dataset) - train_size - test_size
train_dataset, test_size, val_dataset = random_split(dataset, [train_size, test_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


720,000 training samples
160,000 validation samples


In [32]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order (RandomSampler)
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially (SequentialSampler)
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [33]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top; ensure the model does NOT outputs attentions and hidden_states 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [34]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, eps = 1e-8 )

In [35]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = epochs * len(train_dataloader)

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [36]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [37]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [38]:
checkpoint = torch.load("/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/modelcheckpoint.pt")
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_i = checkpoint['epoch']

print(checkpoint['model_state_dict'])
print(checkpoint['optimizer_state_dict'])
print(checkpoint['epoch'])



Streaming output truncated to the last 5000 lines.
        [1.5210e-10, 5.8207e-10, 5.7748e-10,  ..., 4.7458e-10, 1.2746e-09,
         3.3485e-10]], device='cuda:0')}, 172: {'step': tensor(9600.), 'exp_avg': tensor([ 1.0621e-07,  4.7871e-06,  4.2595e-06, -3.1734e-06,  5.1260e-06,
        -6.0066e-06,  9.8131e-06,  1.9550e-05, -7.6035e-06,  2.5529e-07,
         3.0697e-06, -1.3188e-05, -3.5892e-06, -5.3574e-06,  3.7678e-06,
         1.1163e-06, -1.3727e-05,  3.9218e-06,  7.7824e-08, -1.5002e-06,
        -5.1604e-06,  3.7747e-07, -6.7612e-06, -1.1861e-05,  1.2317e-05,
        -8.9324e-06, -1.6490e-07, -4.3514e-07,  2.4563e-06,  1.7085e-06,
         6.2199e-06, -2.5317e-06, -7.2366e-06, -4.8371e-06, -7.9358e-06,
         1.7836e-06,  2.2805e-06,  4.0898e-06,  5.2985e-06,  2.7836e-06,
        -1.5941e-06,  1.0184e-06,  1.5388e-05, -2.2812e-06, -1.6217e-05,
        -9.8295e-06, -1.4933e-05,  2.7068e-06, -1.8671e-06, -9.2581e-06,
        -4.2080e-06,  7.9814e-06, -1.5345e-06,  7.3849e-06,  3

In [ ]:
# WITH CHECKPOINT SAVING

import random
import numpy as np

checkpoint_num = 0 
seed_val = 28
load_from_checkpoint = False

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()


if load_from_checkpoint:
  checkpoint = torch.load('/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/modelcheckpoint.pt')
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  loss = checkpoint['loss']


# For each epoch...
for epoch_i in range(0, epochs):

    if load_from_checkpoint:
      epoch_i = checkpoint['epoch']
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        if load_from_checkpoint:
          step = checkpoint['step']
          batch = checkpoint['batch']
          load_from_checkpoint = False

        # Progress update every 100 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            temp = {'model_state_dict': model.state_dict(),
                        'epoch': epoch_i,
                        'optimizer_state_dict': optimizer.state_dict(),
                        'step':step,
                        'batch':batch,
                        'loss':total_train_loss}
            tempname = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/checkpoint_num_" + str(checkpoint_num) + "_modelcheckpoint.pt"
            torch.save(temp, tempname)

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].float().to(device)
        b_labels = batch[2].float().to(device)

        # 0 the gradients
        model.zero_grad() 

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.       
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask= b_input_mask, 
                       labels= b_labels,
                       return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
        
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end
        total_train_loss += loss.item()
        
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].float().to(device)
        b_input_mask = batch[1].float().to(device)
        b_labels = batch[2].float().to(device)
        
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

torch.save(model.state_dict(), "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/savedfinalmodel.pt")
# yo make sure to rename .pth to .pt after saved


======== Epoch 1 / 2 ========
Training...
  Batch   100  of  22,500.    Elapsed: 0:01:07.
  Batch   200  of  22,500.    Elapsed: 0:02:21.
  Batch   300  of  22,500.    Elapsed: 0:03:36.
  Batch   400  of  22,500.    Elapsed: 0:04:51.
  Batch   500  of  22,500.    Elapsed: 0:06:06.
  Batch   600  of  22,500.    Elapsed: 0:07:21.
  Batch   700  of  22,500.    Elapsed: 0:08:37.
  Batch   800  of  22,500.    Elapsed: 0:09:53.
  Batch   900  of  22,500.    Elapsed: 0:11:08.
  Batch 1,000  of  22,500.    Elapsed: 0:12:24.
  Batch 1,100  of  22,500.    Elapsed: 0:13:39.
  Batch 1,200  of  22,500.    Elapsed: 0:14:54.
  Batch 1,300  of  22,500.    Elapsed: 0:16:10.
  Batch 1,400  of  22,500.    Elapsed: 0:17:25.
  Batch 1,500  of  22,500.    Elapsed: 0:18:40.
  Batch 1,600  of  22,500.    Elapsed: 0:19:56.
  Batch 1,700  of  22,500.    Elapsed: 0:21:11.
  Batch 1,800  of  22,500.    Elapsed: 0:22:27.
  Batch 1,900  of  22,500.    Elapsed: 0:23:42.
  Batch 2,000  of  22,500.    Elapsed: 0:24:5

In [ ]:
# WITHOUT MODEL CHECKPOINT SAVING

import random
import numpy as np

seed_val = 28
load_from_checkpoint = True

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()


if load_from_checkpoint:
  checkpoint = torch.load('/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/modelcheckpoint.pt')
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  loss = checkpoint['loss']


# For each epoch...
for epoch_i in range(0, epochs):

    if load_from_checkpoint:
      epoch_i = checkpoint['epoch']
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        if load_from_checkpoint:
          step = checkpoint['step']
          batch = checkpoint['batch']
          load_from_checkpoint = False

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].float().to(device)
        b_labels = batch[2].float().to(device)

        # 0 the gradients
        model.zero_grad() 

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.       
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask= b_input_mask, 
                       labels= b_labels,
                       return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
        
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end
        total_train_loss += loss.item()
        
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].float().to(device)
        b_input_mask = batch[1].float().to(device)
        b_labels = batch[2].float().to(device)
        
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

torch.save(model.state_dict(), "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/savedfinalmodel.pt")
# yo make sure to rename .pth to .pt after saved


======== Epoch 1 / 2 ========
Training...
  Batch 1,440  of  47,500.    Elapsed: 0:00:00.
  Batch    40  of  47,500.    Elapsed: 0:00:29.
  Batch    80  of  47,500.    Elapsed: 0:00:55.
  Batch   120  of  47,500.    Elapsed: 0:01:23.
  Batch   160  of  47,500.    Elapsed: 0:01:50.
  Batch   200  of  47,500.    Elapsed: 0:02:17.
  Batch   240  of  47,500.    Elapsed: 0:02:46.
  Batch   280  of  47,500.    Elapsed: 0:03:14.
  Batch   320  of  47,500.    Elapsed: 0:03:42.
  Batch   360  of  47,500.    Elapsed: 0:04:10.
  Batch   400  of  47,500.    Elapsed: 0:04:39.
  Batch   440  of  47,500.    Elapsed: 0:05:07.
  Batch   480  of  47,500.    Elapsed: 0:05:35.
  Batch   520  of  47,500.    Elapsed: 0:06:04.
  Batch   560  of  47,500.    Elapsed: 0:06:32.
  Batch   600  of  47,500.    Elapsed: 0:07:00.
  Batch   640  of  47,500.    Elapsed: 0:07:29.
  Batch   680  of  47,500.    Elapsed: 0:07:57.
  Batch   720  of  47,500.    Elapsed: 0:08:25.
  Batch   760  of  47,500.    Elapsed: 0:08:5

KeyboardInterrupt: ignored